# Dimensionality Reduction
$\textbf{Introdution.}$ We have seen that the second step of statistics may be data description. It may be intricately linked to data cleaning: for instance, you realize your data needs cleaning when describing it through a histogram and ending up with absurd values.

On the other hand, dimensionality reduction may serve different purposes:
    
- $\textbf{Visualization:}$ if your initial data lies in a high dimensional space, you may want to visualize it
in $\mathbb{R}^2$ or $\mathbb{R}^3$ or to detect possible patterns or anomalies;

- $\textbf{Providing a suitable input:}$ if your database is so large that you cannot practically compute
estimates with it all, dimensionality reduction is mandatory;

- $\textbf{Avoiding the curse of dimensionality:}$ this expression refers to range of issues encountered
when dealing with data in high-dimensional spaces. Indeed, it becomes more difficult to
detect patterns overall, as data density decreases and computational complexity increases.
Therefore dimensionality reduction techniques also help providing better suited inputs for
models.

$\textbf{Dataset.}$ In this session, we are first focusing on a standard dataset in machine learning: the MNIST dataset of handwritten digits. It contains 70,000 black and white $28\times 28$ pixel images representing handwritten digits from 0 to 9. We are going to answer the following questions:
- How to visualize MNIST images in $\mathbb{R}^2$ in a pertinent way (ie in way that a reflects intrinsic similarities) ?
- How to assess whether dimensionality reduction has yielded qualitatively suitable outputs ?

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import time
import tensorflow as tf
import matplotlib.pyplot as plt
%inline matplotlib

In [ ]:
((X,Y),(enval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()
X=X/np.float32(255)
Y=Y.astype(np.int32)

eval_data = eval_data/np.float32(255)
eval_labels = eval_labels.astype(np.int32)

print("Training data columns "+str(X.shape[0])+"example of size "+str(X.shape[1])+"*"+str(X.shape[2]))
print("Test data contains "+str(eval_data.shape[0])+"example of size"+str(eval_data.shape[1])+"*"+str(eval_data.shape[2]))

### 1) Visualizing data examples: show how data look like, choose one example and show it

In [ ]:
idx = np.random.randint(0,X.shape[0])
print("image has labels"+str(Y[idx]))
plt.figure()
plt.imshow(X[idx])
plt.show()

### 2) Turn data into dataframe
Give names pixel0, ... pixel783 to your columns

In [ ]:
X = np.reshape(X,(X.shape[0],28*28))
feat_cols = ['pixel'+str(i) for i in range(X.shape[1])]
df = pd.DataFrame(X,columns=feat_cols)
df['label'] = Y
df['label'] = df['label'].apply(lambda i: str(i))
print('Size of the dataframe is {}'.format(df.shape))
df.head()

### 3) Using this format, we can visualize a subsample of the digits.
visualizate table with size $3\times 10$, consisting of random digits

In [ ]:
# permutation for random subsampling
rndperm = np.random.permutation(df.shape[0])
# plot some chosen examples
plt.gray()
fig = plt.figure(figsize=(16,7))
for i in range(0,30):
  ax = fig.add_subplot(3,10,i+1,title='Digit:'+df.loc[rndperm[i],"label"]) 
  ax.matshow(df.loc[rndperm[i],feat_cols].values.reshape((28,28)).astype(float))
  ax.set_yticklabels([])
  ax.set_xticklabels([])
plt.show()

### Dimensionality Reduction through Principal Component Analysis (PCA).

The aim of PCA is to turn a set of possibly correlated variables into linearly uncorrelated variables (called “principal components”) through an orthogonal transformation. The idea is that:
- the first component ($C_1$) accounts for most of the variance in the data;
- the second ($C_2$) accounts for the second highest variance, and is orthogonal to $C_1$;
- etc.
The orthogonality condition makes the data way easier to visualize, while the variance criterion ranks components by order of relative importance.

In order to achieve that, the optimization procedure aims at finding a matrix of weights $W$ mapping each $x_i \in \mathbb{R}^d$ to a principal components score $t_{k,i}=\langle x_i, w_k\rangle$ inheriting the most variance from $X$, ie the first weight vector $w_1$ solves: $$\max_{\lvert\lvert w\rvert \rvert} \sum_i (t_{1,i})^2.$$ Other weights are retrieved by the same operation on $X$ minus its first components.

This process yields a decomposition of the form $T=XW$. Using this: 
- dimensionality reduction can be performed in this context by only selecting the first $L$ components: $T_L=X W_L$;
- a first idea for visualization in $\mathbb{R}^2$ is to only represent the first two components.

Let us try this out on the MNIST dataset.

### 4) Perform PCA
Create 2-d scatter plot with first and second dimensions of PCA, draw different dots with different colors. Use `seaborn` library for this.

In [ ]:
pca = PCA(n_components=3)
pca_result=pca.fit_transform(df[feat_cols].values)
df['pca-one'] = pca_result[:,0]
df['pca-two'] = pca_result[:,1]
df['pca-three'] = pca_result[:,2]
print('Explained variance ratio per principal component:{}'.format(pca.explained_variance_ratio_))
import seaborn as sns

sns.pairplot(x_vars=['pca-one'],y_vars=['pca-two'],data=df,hue='label',height=6)

### 5) Make inference about this method
It seems that 0s and 1s are well separated from other classes. However, for the other labels, let us try to do better.

### Dimensionality Reduction through t-distributed stochastic neighbor embedding (t-SNE)

On the other hand, we study t-SNE, which is a nonlinear dimensionality reduction method particularly suited for the visualization of high-dimensional data.

The idea is reduce dimensionality by minimizing the distance between pairwise probabilities in the high-dimensional space and in a lower-dimensional space. It consists of two steps:
- constructing a probability on the high dimensional object pairs: similar pairs have a high probability of being chosen, unlike dissimilar observations;
- creating a similar probability distribution in a low dimensional space, minimizing some distance between the two.

Let us try it: is it possible to apply it on all the data? Does that improve the visual performance?

### 6) Use t-SNE model to do dimensionality reduction and plot results

In [ ]:
# T-sne
n_sne = 7000 #data subsampling
time_start = time.time()
tsne = TSNE(n_components=2,verbose=1,perplexity=40,n_iter=300)
tsne_result = tsne.fit_transform(df.loc[rndperm[:n_sne],feat_cols].values)
print('t-SNE done! Time elapsed: {}seconds'.format(time.time()-time_start))
df_tsne = df.loc[rndperm[:n_sne],:].copy()
df_tsne['x-tsne'] = tsne_result[:,0]
df_tsne['y-tsne'] = tsne_result[:,1]

import seaborn as sns
sns.pairplot(x_vars=['x-tsne'],y_vars=['y-tsne'],data=df_tsne,hue='label',height=6)

### 7) Make inference about this method
On a subsample, we seem to have gained performance. However some classes are still quite difficult to differenciate. Let us see if we can do even better...

### Combination of the two.

Since PCA handles large inputs better, a solution would be to combine the two, and perform t-SNE on PCA outputs. How does that improve performance ? How far can we go in terms of components ?

### 8) use 50 dimentions with PCA model

In [ ]:
# T-sne on larger PCA
pca_50 = PCA(n_components=50)
pca_result_50 = pca_50.fit_transform(df[feat_cols].values)
print('Cumulative explianed variation for 50 principal components: {}'.format(np.sum(pca_50.explained_variance_ratio)))

In [ ]:
n_sne = 10000
time_start = time.time()
tsne = TSNE(n_components=2,verbose=1,perplexity=40,n_iter=300)
tsne_pca_results = tsne.fit_transform(pca_result_50[rndperm[:n_sne]])
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

### 9) Implement the tSNE model and visualise results and make inference

In [ ]:
df_tsne = None
df_tsne = df.loc[rndperm[:n_sne],:].copy()
df_tsne['x-tsne-pca'] = tsne_pca_results[:,0]
df_tsne['y-tsne-pca'] = tsne_pca_results[:,1]

sns.pairplot(x_vars=['x-tsne-pca'],y_vars=['y-tsne-pca'],data=df_tsne,hue='label',height=6)